## **Simulation Set-Up**

In [1]:
import os

import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    Parser,
    ProcessModelDirectives,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
)
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer

from manipulation import running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import ConfigureParser

In [2]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

# Useful links mentionned 

A great resource is [Authoring a Multibody Simulation tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/authoring_multibody_simulation-add293478aac40a984845aa3705eefdd?). Parts of this notebook were taken from it.

Here are two useful resources for [URDF](https://wiki.ros.org/urdf/Tutorials/Building%20a%20Visual%20Robot%20Model%20with%20URDF%20from%20Scratch) and [SDFormat](https://classic.gazebosim.org/tutorials?tut=build_model) creation. ([Another SDF reference](http://sdformat.org/spec))

Meshlab](https://www.meshlab.net/), an open-source software, is a handy tool to convert common formats to a .obj. 

A note about meshes. Support for meshes as collision geometry is limited as documented [here](https://drake.mit.edu/doxygen_cxx/group__geometry__file__formats.html). In certain cases--point contact, compliant hydroelastic contact, checking collision candidates, signed distance queries--the mesh will be represented by its [convex hull](https://en.wikipedia.org/wiki/Convex_hull) instead. You can convert meshes into sdf's using [manipulation/create_sdf_from_mesh.py](https://github.com/RussTedrake/manipulation/blob/master/manipulation/create_sdf_from_mesh.py), which additionally supports convex decomposition of a nonconvex mesh and mesh simplification.

### Loading and Viewing Models

Drake has some pre-existing models which can be found in the [drake_models repo](https://github.com/RobotLocomotion/models) and in the [manipulation models](https://github.com/RussTedrake/manipulation/tree/master/manipulation/models). You might find some of these useful for your final project!

Typically to add a model to a MultibodyPlant, you will access its [Parser](https://drake.mit.edu/pydrake/pydrake.multibody.parsing.html#pydrake.multibody.parsing.Parser) object ([C++ documentation](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_parser.html)). 

Drake provides a `ModelVisualizer` class to visualize models interactively. This will help as we start to produce our own robot description files, or port description files over from another simulator. In the next few cells, we will go over different ways to add simulated objects or models to your simulation. 


For simpler objects and projects, sometimes it can be quicker to create your own model from scratch in a string or a new file as above. However, you can also find some 3D models online. Model files can be in any of the formats supported by the [Parser](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_parser.html), though SDF is recommended. Places you can find these models:

- [Drake examples](https://github.com/RobotLocomotion/drake/tree/master/examples)

- Object databases curated for robotics such as YCB. Some are already in drake_models [here](https://github.com/RobotLocomotion/models/tree/master/ycb)!

- [Rubik's Cube](https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/02-Lets-get-you-a-robot-8f86172b-b597-4ceb-9bad-92d11ac7a6cc/notebook/rubiks_cube-35164353b13d4a47910ca14e588c74d6?)

- It is also worth checking [final projects](https://manipulation.csail.mit.edu/misc.html#projects) that had objects you'd like to use, see if their code is open source or ask!

- [TurboSquid](https://www.turbosquid.com/3d-model/free): will often need converting to an .obj file. The .obj file can then be directly turned into a SDF file by using the below terminal command, documented [here](https://drake.mit.edu/pydrake/pydrake.multibody.mesh_to_model.html). On Deepnote, you can open a terminal on the left bar and run the command after you have uploaded your .obj file. 
```sh
python3 -m pydrake.multibody.mesh_to_model --scale=1.0 path/to/mesh.obj
```

## Creating a Simulation

So far we've gone over how to create and examine individual objects. A useful scene will usually have multiple bodies within it. You can put each one in its own model file and load them in one by one to the plant as in the function below. 

We weld the table top to the world frame and set the poses for each free body so they fall onto the table when the simulation starts. Note that we are recording the scene, so you can replay the scenario using the control panel!

Alternatively, you can define a [Model Directives](https://github.com/RobotLocomotion/drake/blob/master/multibody/parsing/README_model_directives.md) file. This is a YAML file that defines all your objects in the simulation and weld frames as needed to describe relative poses of your objects. In the code cell below, we define a model directive that does the same as the cell above and loads it into the plant using `LoadModelDirectivesFromString` and `ProcessModelDirectives`[(docs)](https://drake.mit.edu/doxygen_cxx/process__model__directives_8h.html). While not necessary here, it is a common final project setup to use scenarios and HardwareStation.

In [3]:
# NOTE: I use a file here to avail you of the fact that you can do this
# It is preferable to use the dummy project package we created earlier
# to reference the table_top because using explicit files will need absolute paths.
model_directive = f"""
    directives:
    - add_model:
        name: table_top
        file: file://{os.getcwd()}/table_top.sdf
    - add_weld:
        parent: world
        child: table_top::table_top_center
    - add_model:
        name: cracker_box
        file: package://drake_models/ycb/003_cracker_box.sdf
        default_free_body_pose:
            base_link_cracker:
                translation: [0,0,0.8]
                rotation: !Rpy {{ deg: [42, 33, 18] }}    
    - add_model:
        name: sugar_box
        file: package://manipulation/hydro/004_sugar_box.sdf
        default_free_body_pose:
            base_link_sugar:
                translation: [0,-0.25,0.8]
    """


def create_scene_directives(model_directive, sim_time_step=0.001):
    meshcat.Delete()
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=sim_time_step)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.package_map().Add("dummy_project", os.path.abspath(""))

    directives = LoadModelDirectivesFromString(model_directive)
    models = ProcessModelDirectives(directives, plant, parser)
    plant.Finalize()

    AddDefaultVisualization(builder=builder, meshcat=meshcat)

    diagram = builder.Build()
    return diagram


def run_simulation(sim_time_step):
    diagram = create_scene_directives(model_directive, sim_time_step)
    simulator = Simulator(diagram)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0)
    meshcat.PublishRecording()


run_simulation(0.001)

Using the same YAML you can also create a manipulation scenario to eventually use as a `HardwareStation`. This gives you more tools to work with manipulation setups and have a common project base as most of our questions and examples from the course etc. Using a scenario you can also define the timestep and what kind of contact model you want for your scene (set using the directives file or the [`scenario.plant_config.contact_model`](https://drake.mit.edu/doxygen_cxx/structdrake_1_1multibody_1_1_multibody_plant_config.html)).

In [ ]:
# A rough sketch of TODOs to get you started:

# 1. Create or save an object in a file path/string/package url.
#       (Visualizing your model to test it first might be helpful)



# 2. Add your objects to a plant/diagram
# 3. Run a simulator object on this diagram
# try changing the directives string above to have hydroelastic contact properties!


scene = open("/work/manipulation/project/objects/environment_setup.yaml")

if os.getcwd() == "/datasets/_deepnote_work": 
    scene = open("/work/manipulation/project/objects/environment_setup.yaml")
    xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
else:
    scene = open("objects/environment_setup.yaml") # local setup
    xmls = [os.getcwd() + "/package.xml"]


def preview_scene_given_directives(model_directive):
    meshcat.Delete()
    scenario = LoadScenario(data=model_directive)
    station = MakeHardwareStation(
        scenario, meshcat, package_xmls=xmls
    )

    simulator = Simulator(station)
    context = simulator.get_mutable_context()
    x0 = station.GetOutputPort("mobile_iiwa.state_estimated").Eval(context)
    station.GetInputPort("mobile_iiwa.desired_state").FixValue(context, x0)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


preview_scene_given_directives(scene)

# hello there

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=92fbdad2-a4a9-4ef8-b493-c307060b34a1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>